# Emily Farr - Observing with `astroplan`

* I have given you three locations, some constraints, and list of targets: `TargetStars.csv`.
* You are observing on June 21, 2018.
* Your job is to determine which of the three locations can see the most targets, given the constraints.
* Points awarded will depend on how clearly you support your answer.

# Date: June 21, 2018.

---

# Locations

- Grytviken, South Georgia Island
- Bairiki, South Tarawa, Kiribati
- Lochboisdale, Outer Hebrides, Scotland

---

# Constraints

- Observing at (local) night
- Airmass < 2
- Moon Separation > 30 degrees

---

In [1]:
import numpy as np
from astropy.table import QTable
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord, EarthLocation
import pytz

from astroplan import Observer, FixedTarget

import warnings
warnings.filterwarnings('ignore', category=Warning)
from astroplan import AltitudeConstraint, AirmassConstraint
from astroplan import observability_table
from astroplan import moon_illumination
from astroplan import MoonSeparationConstraint
%matplotlib inline
from astroplan import time_grid_from_range

from astroplan.plots import plot_sky, plot_airmass
import numpy as np
import matplotlib.pyplot as plt


>>> from astroplan import download_IERS_A
>>> download_IERS_A()
 [astroplan.utils]


In [2]:
target_table = QTable.read('TargetStars.csv', format='ascii.csv')
target_table

Name,RA,DEC
str15,float64,float64
Mizar,13.39872773,54.92541525
Spica,13.41989015,-11.16124491
Alkaid,13.79237392,49.31330288
Hadar,14.06373459,-60.3729784
Arcturus,14.2612076,19.18726997
RigelKentaurusB,14.66094188,-60.83947139
RigelKentaurusA,14.66136068,-60.83514707
Izar,14.74979191,27.07417383
Kochab,14.84510983,74.15547596


In [48]:
observe_date= Time("2018-6-21 0:00:00",format='iso')
Grytviken = Observer(latitude = -54 * u.deg + 16 * u.arcmin + 53.4 * u.arcsec,
                    longitude = -36 * u.deg + 30 * u.arcmin + 28.8 * u.arcsec,
                    timezone = 'Atlantic/South_Georgia',
                    name = "Grytviken"
                    )
Bairiki = Observer(longitude = 172.979,
                    latitude = 1.3291,
                    timezone = 'Pacific/Tarawa',
                    name = "Bairiki"
                    )
Lochboisdale = Observer(longitude = -7.3167 * u.deg + 154 * u.arcmin + 53.4 * u.arcsec,
                    latitude = 57.1667 * u.deg + 30 * u.arcmin + 28.8 * u.arcsec,
                    timezone = 'GMT',
                    name = "Lochboisdale"
                    )

In [49]:
targets = [FixedTarget(coord=SkyCoord(ra = RA*u.hourangle, dec = DEC*u.deg), name=Name)
           for Name, RA, DEC in target_table]

In [61]:
Grytviken_start = Grytviken.sun_set_time(observe_date, which='nearest')
Grytviken_end = Grytviken.sun_rise_time(observe_date, which='next')
Bairiki_start = Bairiki.sun_set_time(observe_date, which='nearest')
Bairiki_end = Bairiki.sun_rise_time(observe_date, which='next')
Lochboisdale_start = Lochboisdale.sun_set_time(observe_date, which='nearest')
Lochboisdale_end = Lochboisdale.sun_rise_time(observe_date, which='next')

Grytviken_range = [Grytviken_start,Grytviken_end]
Bairiki_range = [Bairiki_start,Bairiki_end]
Lochboisdale_range = [Lochboisdale_start,Lochboisdale_end]

a = Grytviken_observing_length = (Grytviken_end - Grytviken_start).to(u.h)
b = Bairiki_observing_length = (Bairiki_end - Bairiki_start).to(u.h)
c = Lochboisdale_observing_length = (Lochboisdale_end-Lochboisdale_start).to(u.h)

print("You can observe for {0:.1f} that night at Grytviken, {1:.1f} at Bairiki and {2:.1f} at Lochboisdale.".format(a,b,c))

You can observe for 16.8 h that night at Grytviken, 11.9 h at Bairiki and 6.2 h at Lochboisdale.


In [59]:
constraints = [AltitudeConstraint(20*u.deg, 80*u.deg)]
constraints.append(AirmassConstraint(2))
moon_illumination(Grytviken_start)
constraints.append(MoonSeparationConstraint(30*u.deg))

In [111]:
Grytviken_table = observability_table(constraints, Grytviken, targets, time_range=Grytviken_range)
f = sum([Grytviken_table['ever observable']])
f.sum()

9

In [112]:
Bairiki_table = observability_table(constraints, Bairiki, targets, time_range=Bairiki_range)
e = sum([Bairiki_table['ever observable']])
g = e.sum()
g

21

In [99]:
Lochboisdale_table = observability_table(constraints, Lochboisdale, targets, time_range=Lochboisdale_range)
d = sum([Lochboisdale_table['ever observable']])
d.sum()

18

In [110]:
print("You can observe the most targets on June 21st 2018, {0:.1f}, at Bairiki.".format(g))

You can observe the most targets on June 21st 2018, 21.0, at Bairiki.


### Due Tue Nov 14 - 5pm
- `Make sure to change the filename to your name!`
- `Make sure to change the Title to your name!`
- `File -> Download as -> HTML (.html)`
- `upload your .html and .ipynb file to the class Canvas page`  